# Base Case

In [83]:
# Feb 23
from fastai.tabular import *
from fastai.metrics import fbeta
from tqdm import tqdm

In [84]:
df = pd.read_csv('../data/with_lith.csv', index_col=0)

# Load data + Preprocess

In [85]:
df.columns

Index(['DENS', 'DTS', 'GR', 'PEF', 'RESD', 'RESM', 'RESS', 'NEUT', 'SP',
       'lith'],
      dtype='object')

In [86]:
df.head()

,DENS,DTS,GR,PEF,RESD,RESM,RESS,NEUT,SP,lith
1717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sand
1718,2.137,NaN,84.724,2.262,78.705,58.731,7.512,0.429,-100.354,Sand
1719,2.104,NaN,83.405,2.396,75.329,58.202,7.667,0.408,-101.271,Sand
1720,2.136,NaN,83.974,2.399,80.198,61.827,7.817,0.388,-102.063,Sand
1721,2.117,NaN,88.314,2.278,83.477,63.515,8.927,0.421,-102.805,Sand


In [87]:
labels = df.lith.unique()

In [88]:
dep_var = 'lith'
cat_names = []
cont_names = ['DENS', 'DTS', 'GR', 'PEF', 'RESD', 'RESM', 'RESS', 'NEUT', 'SP']
procs = [FillMissing, Categorify, Normalize]

In [89]:
test = TabularList.from_df(df, path='.', cat_names=cat_names, cont_names=cont_names)

In [90]:
data = (TabularList.from_df(df, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .random_split_by_pct()
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [91]:
learn = tabular_learner(data, layers=[200,100], metrics=[accuracy])

In [92]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.064414,0.041068,0.983704


In [124]:
preds,y = learn.get_preds()

In [94]:
type(np.array(preds))

numpy.ndarray

In [116]:
lala = pd.DataFrame(data=[result,y])

In [122]:
lala = lala.T

,0,1
0,Sand,Sand
1,Sand,Sand
2,Sand,Sand
3,Sand,Sand
4,Shale,Shale


In [104]:
preds[0:5]

tensor([[9.9999e-01, 1.0277e-05],
        [9.9422e-01, 5.7780e-03],
        [1.0000e+00, 1.9051e-06],
        [1.0000e+00, 6.4910e-10],
        [2.6580e-01, 7.3420e-01]])

In [115]:
y = list(map(lambda x:labels[x.item()],y))

In [125]:
result = list(map(lambda x:labels[np.argmax(x).item()],preds)).T

AttributeError: 'list' object has no attribute 'T'

In [81]:
result[0:5]

['Sand', 'Sand', 'Sand', 'Sand', 'Sand']

In [25]:
preds = []
for i in tqdm(range(len(df))):
#     print(i,)
    row = df.iloc[i]
    p = learn.predict(row)
#     print(p[0])
    preds.append(str(p[0]))

  0%|          | 556/242088 [00:15<2:00:55, 33.29it/s]

KeyboardInterrupt: 

  0%|          | 556/242088 [00:30<2:00:55, 33.29it/s]